## Zhlukovanie - Vyhodnotenie a interpretácia zhlukov - príklad 1

Nasledujúci príklad demonštruje zhlukovanie nákupných dát ako v minulotýždňovom príklade. 

Tentoraz sa zameriame na iné kritériá pre vyhodnotenie kvality a kompaktnosti zhlukov a ich interpretáciu nie pomocou vizualizácií, ale pomocou rozhodovacích stromov. 

Najprv si importujeme potrebné knižnice pre prácu s dátovými rámcami, poľami a pre vykresľovanie grafov. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Do dátového rámca načítame vstupné dáta zo súboru. Vypíšeme prvých 5 záznamov.  

In [ ]:
data = pd.read_csv('../data/wholesale.csv')
data.head()

Rovnako ako v predošlom cvičení transformujeme dáta pomocou One Hot Encoderu (oba atribúty obsahujúce kategorické dáta) a 5 prvých záznamov transformovaného datasetu vypíšeme na obrazovku. 

In [ ]:
data = pd.get_dummies(data, columns=['Channel', 'Region']) 
data.head()

Keďže budeme vytvárať K-Means model, tak numerické atribúty normalizujeme použitím MinMaxScaler-u. 

In [ ]:
from sklearn.preprocessing import MinMaxScaler # importujeme MinMaxScaler

scaler = MinMaxScaler() # Inicializujeme transformátor
scaler.fit(data) # aplikujeme ho na vstupné dáta

# po aplikovaní scaleru budeme mať výstup vo forme numpy poľa
# to môžeme - ale nemusíme - naspať transformovať do pandas rámca (ak chceme ešte robiť nejaké predspracovanie)
# funkcie pre trénovanie modelov potom vedia pracovať aj s pandas aj s numpy

# data_norm = scaler.transform(data)
data_norm = pd.DataFrame(scaler.fit_transform(data), index=data.index, columns=data.columns)

#### Kritérium Silhouette 

Okrem sumy štvorocov vzdialeností od reprezentanta zhluku môžeme použiť viacero iných metrík, ktoré definujú kvalitu jednotlivých zhlukov. Tie má zmysel použiť:
* tam, kde nepoužívame metódy, ktoré vytvárajú reprezentantov zhlukov
* vtedy, ak chceme použiť iné kritérium, ako používa samotný algorimtus 

Jedným z takýchto kritérií je index Silhouette. Ten udáva koeficient, vypočítaný pre každý príklad a spriemernený pre celú dátovú množinu. Koeficient kombinuje priemernú hodnotu metriky vnútro-zhlukovej vzdialenosti s priemernou vzdialenosťou k najbližšiemu zhluku. Koeficient nadobúda hodnoty od 0 a 1 (pre každý príklad). Hodnota blízka nule znamená, že príklad je pravdepodobne zaradený do nesprávneho zhluku a hodnoty bližšie k 1 vyjadrujú, že príklad je regulérnym prvkom predikovaného zhluku a dobre odlíšiteľný od ostatných. Koeficient Silhouette v scikit-learn potom vypočítava priemernú hodnotu pre všetky príklady. To potom umožňuje porovnať viacero zhlukovacích modelov (s rôznymi počtami zhlukov) navzájom. 

Podobne ako v prípade hľadania správnej hodnoty zhlukov pomocou sumy štvorcov vzdialeností, môžeme v cykle povytvárať viacero modelov, ktoré prostredníctvom tohoto kritéria evaluujeme.

In [ ]:
from sklearn.cluster import KMeans # importujeme knižnicu pre KMeans
from sklearn.metrics import silhouette_score # importujeme funkciu pre výpočet Silhouette

# použijeme Silhouette score pre počet zhlukov
# môžeme potom porovnať ideálne počty zhlukov pre rôzne kritériá

K = range(2,10) # vygenerujeme pole parametrov (počet zhlukov)

results = [] 

# v cykle vytvoríme pre každú hodnotu parametra zhlukovací model, počet zhlukov zodpovedá hodnote iterátora

for k in K:
    model = KMeans(n_clusters = k)
    model.fit(data_norm)
    predictions = model.predict(data_norm) # pre výpočet silhouette priradíme príklady zo vstupných dát do zhlukov
    results.append(silhouette_score(data_norm, predictions)) # vypočítame skóre a priradíme ho do znoznamu, v ktorom budeme ukladať všetky skóre

In [ ]:
# výsledky môžeme vypísať na obrazovku
# zoznam obsahuje Silhouette skóre pre parametre, v poradí, v akom boli vytvárané

print(results)

Ak chceme, skóre Silhouette vieme vizualizovať rovnakým spôsobom ako v prípade sumy štvorcov vzdialeností od centroidu. 

### Úloha 13.1:

Použite matplotlib rovnako ako v úlohách z predošlého cvičenia na vykreslenie závislosti počtu zhlukov od Silhouette skóre.

In [ ]:
# YOUR CODE HERE

Teraz môžeme natrénovať model s najlepšim skóre. 

In [ ]:
model = KMeans(n_clusters=6) # vytvoríme model pre stanovený počet zhlukov
model.fit(data_norm) # naučíme na trénovacej množine

labels = model.predict(data_norm) # vstupné dáta zatriedime do zhlukov

In [ ]:
# na príslušnosť príkladov do zhlukov sa môžeme pozrieť vypísaním ich predikcií
print(labels)

Podstatnou informáciou môže byť aj početnosť jednotlivých zhlukov v rámci vstupných dát. Tú si môžeme jednoducho spočítať z `labels` a to tak, že spočítame počty výskytov rôznych prvkov poľa výsledkov zhlukovania.

In [ ]:
clusters, counts = np.unique(labels, return_counts=True) # pomocou funkcie unique identifikujeme rôzne hodnoty a vrátime aj ich počty
print(np.asarray((clusters, counts))) # aby sme "krajšie" naformátovali výstup, spojíme ich do numpy poľa

### Interpretácia zhlukov pomocou klasifikátorov

Jednou z možností (okrem skúmania hodnôt atribútov atď.), ako interpretovať výsledné zhluky je postaviť nad danými zhlukmi klasifikačné modely, ktoré umožnia príklady patriace do daného zhluku nejakým spôsobom popísať.

V takomto prípade je proces nasledovný - zhlukovaním si vlastne z pohľadu klasifikácie "vygenerujeme" cieľový atribút. Jednotlivé zhluky potom v podstate predstavujú jeho jednotlivé hodnoty - triedy. K vstupným dátam môžeme teda priradiť "cieľový atribút", ktorý ale teraz vyjadruje príslušnosť príkladu do konkrétneho zhluku. Nad takýmito dátami teda môžeme vytvoriť klasifikačný model - ideálne taký, ktorý je dobre reprezetovateľný a pochopiteľný, keďže našim cieľom je vytvorené zhluky pochopiť a porozumieť im, ideálne aj popísať napr. pomocou kombinácie hodnôt atribútov.

Keď použijeme vstupné dáta (`data`) a vektor príslušnosti príkladov do zhlukov (`labels`), vytvoríme tak v podstate dvojicu matica príznakov a vektor hodnôt cieľového atribútu, ktoré používame v klasifikácii. Dáta sú potom pripravené v takej podobe, že ich môžeme použiť na trénovanie klasifikačných modelov. 

In [ ]:
# dátový rámec data v podstate zodpovedá matici príznakov
# stĺpec hodnôt cieľového atribútu zodpovedá vektoru hodnôt cieľového atribútu

X_train = data
y_train = labels

In [ ]:
print(X_train.shape)
print(y_train.shape)

### Úloha 13.2:

Aký klasifikátor je potrebné natrénovať, na takýchto dátach, aby sme vedeli získať štruktúru vhodnú pre popis zhlukov? Aké znalosti z takéhoto modelu a v akej forme potom môžeme získať?

### Úloha 13.3.:

Natrénujte vhodne zvolený typ modelu na vstupných dátach. V prípade potreby ešte dodatočne predspracujte dáta. Zvoľte metódu pre nájdenie parametrov, alebo parametre modelu odhadnite.

In [ ]:
# YOUR CODE HERE

### Úloha 13.4:

Natrénujte model s vhodnými parametrami na vstupných dátach a zobrazte preň `confusion_matrix`. Porovnajte výslednú maticu s výsledkami početnosti jednotlivých zhlukov.  

In [ ]:
# YOUR CODE HERE

### Úloha 13.5:

Použite kód z príkladov z predchádzajúcich cvičení a pokúste sa vizualizovať vytvorený model. Viete pomocou znalostí, ktoré z jeho štruktúry odvodíte popísať jednotlivé triedy, resp. zhluky?

In [ ]:
# YOUR CODE HERE

from sklearn import tree
from sklearn.tree import export_graphviz

with open("file.txt", "w") as f:
    f = tree.export_graphviz(dtree, feature_names=X_train.columns.values, class_names=["0", "1", "2", "3", "4", "5"], out_file=f)